<a href="https://colab.research.google.com/github/thamadhi/telco-customer-churn-CM2604/blob/main/Notebooks/Preprocessing_Task_2_Scaled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
!git clone https://github.com/thamadhi/telco-customer-churn-CM2604.git


fatal: destination path 'telco-customer-churn-CM2604' already exists and is not an empty directory.


In [80]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np


In [81]:
# Load data
data = 'telco-customer-churn-CM2604/data/Telco-Customer-Churn-unprocessed.csv'
df = pd.read_csv(data)

In [82]:
#remove empty strings
df = df[df['TotalCharges'] != ' ']
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

In [83]:
# Convert Yes/No columns to 1/0
yes_no_cols = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity',
               'OnlineBackup','DeviceProtection','TechSupport','StreamingTV',
               'StreamingMovies','PaperlessBilling','Churn','gender']
for col in yes_no_cols:
    if col in df.columns:
        df[col] = df[col].replace({'Yes': 1, 'No': 0, 'Female': 1, 'Male': 0})

/tmp/ipython-input-104487984.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'Yes': 1, 'No': 0, 'Female': 1, 'Male': 0})


In [84]:
# Fix the columns that still have these values
service_cols = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport', 'StreamingTV', 'StreamingMovies']

for col in service_cols:
    if col in df.columns:
        df[col] = df[col].replace(['No internet service', 'No phone service'], 0)

/tmp/ipython-input-3072376738.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(['No internet service', 'No phone service'], 0)


In [85]:
#Drop customer ID
df = df.drop(columns=['customerID'])

In [86]:
# Separate X, y
X = df.drop('Churn', axis=1)
y = df['Churn']

In [87]:
# Identify true categorical columns (non-numeric, not already encoded)
numeric_features = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [88]:
# Get object columns and check data types
categorical_features = []
binary_features = []

for col in X.columns:
    if col not in numeric_features:
        # Check if column is object type (strings) or has more than 2 unique values
        if X[col].dtype == 'object':
            categorical_features.append(col)
        elif X[col].nunique() > 2:
            categorical_features.append(col)
        elif X[col].nunique() == 2:
            # Binary columns, keep as is (already 1/0)
            binary_features.append(col)

print(f"Numeric features: {numeric_features}")
print(f"Categorical features to encode: {categorical_features}")
print(f"Binary features (already encoded): {binary_features}")

# Check data types in categorical features
print("\nChecking data types in categorical features:")
for col in categorical_features:
    print(f"{col}: dtype={X[col].dtype}, unique values={X[col].unique()[:5]}")


Numeric features: ['tenure', 'MonthlyCharges', 'TotalCharges']
Categorical features to encode: ['InternetService', 'Contract', 'PaymentMethod']
Binary features (already encoded): ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']

Checking data types in categorical features:
InternetService: dtype=object, unique values=['DSL' 'Fiber optic' 'No']
Contract: dtype=object, unique values=['Month-to-month' 'One year' 'Two year']
PaymentMethod: dtype=object, unique values=['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']


In [89]:
#Preprocessor: one-hot for categoricals, scale numerics
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Only include categorical features if there are any
if categorical_features:
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
else:
    transformers = [
        ('num', numeric_transformer, numeric_features)
    ]

preprocessor = ColumnTransformer(
    transformers=transformers,
    remainder='passthrough'  # This will pass through binary features
)

In [90]:
# Fit and transform
preprocessor.fit(X)
X_processed = preprocessor.transform(X)

In [91]:
# Get feature names
feature_names = preprocessor.get_feature_names_out()
X_processed_df = pd.DataFrame(X_processed, columns=feature_names)
X_processed_df['Churn'] = y.values


In [92]:
#this file is saved in the git repo under data folder
X_processed_df.to_csv('telco-customer-churn-Scale-PROCESSED.csv', index=False)

from google.colab import files
files.download('telco-customer-churn-Scale-PROCESSED.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>